# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [6]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

629

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [7]:
# Url for call
url = "http://api.openweathermap.org/data/2.5/weather?"

params = {'appid': weather_api_key,
          'units': 'imperial'}

In [ ]:
# Initialize values for for loop - 
# counter to keep track of records for print statement and 
count = 0
# empty list to store weather data to make into
city_weather = []

print(f'Beginning Data Retrieval')
print('-----------------------------')

# for loop to iterate through cities selected
for city in cities:
    print(f'Processing Record {count} | {city.title()}')

    count += 1
    # Set URL paramater for the location
    params['q'] = city.title()
    
    # Exception handling - if city is not in OpenWeather API skip
    try:
        # Build query URL
        weather_response = requests.get(url = url, params = params)
        weather_data = weather_response.json()
        
        # Create dictionary for data from API call
        weather = {}
        
        # Create key's and assign data to it based on API call
        weather['City'] = city
        weather['Lat'] = weather_data['coord']['lat']
        weather['Lng'] = weather_data['coord']['lon']
        weather['Max Temp'] = weather_data['main']['temp_max']
        weather['Humidity'] = weather_data['main']['humidity']
        weather['Cloudiness'] = weather_data['clouds']['all']
        weather['Wind Speed'] = weather_data['wind']['speed']
        weather['Country'] = weather_data['sys']['country']
        weather['Date'] = weather_data['dt']
        # Add dictionary to end of list
        city_weather.append(weather)
    except:
        print(f'City not found. Skipping...')
        continue


print('-----------------------------')
print('Data Retrieval Complete') 
print('-----------------------------')

Beginning Data Retrieval
-----------------------------
Processing Record 0 | Aklavik
Processing Record 1 | Illoqqortoormiut
City not found. Skipping...
Processing Record 2 | Port Elizabeth
Processing Record 3 | Shakawe
Processing Record 4 | Butaritari
Processing Record 5 | Port-Cartier
Processing Record 6 | Hobyo
Processing Record 7 | Vaini
Processing Record 8 | Hermanus
Processing Record 9 | Nizhneyansk
City not found. Skipping...
Processing Record 10 | Mataura
Processing Record 11 | Ushuaia
Processing Record 12 | Longyearbyen
Processing Record 13 | Barrow
Processing Record 14 | Nanortalik
Processing Record 15 | Palmer
Processing Record 16 | Boa Vista
Processing Record 17 | Punta Arenas
Processing Record 18 | Port Alfred
Processing Record 19 | Beloha
Processing Record 20 | Khonuu
City not found. Skipping...
Processing Record 21 | Busselton
Processing Record 22 | Lata
Processing Record 23 | Atuona
Processing Record 24 | Jamestown
Processing Record 25 | Rikitea
Processing Record 26 | Dw

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# Create DataFrame with raw data (list of dictionaries) from API call
city_weather_df = pd.DataFrame(city_weather)

# Display DataFrame
city_weather_df

In [ ]:
# Round Lat and Lng to 2 decimal places
city_weather_df['Lat'] = round(city_weather_df['Lat'],2)
city_weather_df['Lng'] = round(city_weather_df['Lng'],2)

city_weather_df

In [ ]:
# Save DataFrame to .csv
city_weather_df.to_csv('data/city_weather_data.csv')

In [ ]:
# Create and Display summary DataFrame
summary_df = city_weather_df.describe()
summary_df

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
# Initializing count and empty list to store indexes
count = 0
humidity_indexes_to_drop = []

# Find all Indexes with Humidity over 100%
for value in city_weather_df['Humidity']:
    if value > 100 == True:
        humidity_indexes_to_drop.append(count)
        count += 1
    else:
        count += 1

# Display indexes to drop
if humidity_indexes_to_drop == []:
    print('Nothing to drop')
else:
    print(f'Drop indexes {humidity_indexes_to_drop}')


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

#Skipped since no outlier values

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
x_values = city_weather_df['Lat']
y_values = city_weather_df['Max Temp']
plt.scatter(x_values,y_values, edgecolors="black")
plt.title('City Latitude vs. Temperature (09/27/21)')
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.grid(b=True, axis='both')
plt.tight_layout()
plt.savefig("images/lat_temp.png")
plt.show()

The above scatter plot shows the relationship between temperature and latitude. When latidute is equal to 0 you are at the equator which experiences the highest temperatures. Temperatures decrease as you go further North or South. This is likely a negative parabolic relationship.

## Latitude vs. Humidity Plot

In [ ]:
x_values = city_weather_df['Lat']
y_values = city_weather_df['Humidity']
plt.scatter(x_values,y_values, edgecolors="black")
plt.title('City Latitude vs. Humidity (09/27/21)')
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.grid(b=True, axis='both')
plt.tight_layout()
plt.savefig("images/lat_humidity.png")
plt.show()

The above scatter plot shows the relationship between Latitude and humidity. This shows that at any latitude, based on the cities evaluated, you can have a wide range of humidity % with the majority of locations tending towards a higher humid index.

## Latitude vs. Cloudiness Plot

In [ ]:
x_values = city_weather_df['Lat']
y_values = city_weather_df['Cloudiness']
plt.scatter(x_values,y_values, edgecolors="black")
plt.title('City Latitude vs. Cloudiness (09/27/21)')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.grid(b=True, axis='both')
plt.tight_layout()
plt.savefig("images/lat_cloudiness.png")
plt.show()

The above scatter plot shows the relationship between Latitude and cloudiness. This shows that at any latitude, based on the cities evaluated, you can have a wide range of cloudiness. There is a slight tendancy for locations from -40 to -20 and 20 to 40 latitude to have low cloudiness, -20 to 20 and 40 to 60 to have very high cloudiness.

## Latitude vs. Wind Speed Plot

In [ ]:
x_values = city_weather_df['Lat']
y_values = city_weather_df['Wind Speed']
plt.scatter(x_values,y_values, edgecolors="black")
plt.title('City Latitude vs. Wind Speed (09/27/21)')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.grid(b=True, axis='both')
plt.tight_layout()
plt.savefig("images/lat_windspeed.png")
plt.show()

The above scatter plot shows the relationship between Latitude and wind speeds. This shows that across all latitudes, wind speed does not tend to go aboce 25 mph in September for the citites selected. 

## Linear Regression

In [ ]:
# Create bins for different Hemispheres
bins = [-90, -.1, 90]

group_names = ['Southern Hemisphere', 'Northern Hemisphere']

city_weather_df["Hemisphere"] = pd.cut(city_weather_df['Lat'], bins, 
                                  labels=group_names, include_lowest=True)

#Filter by the bins
north_df = city_weather_df[city_weather_df['Hemisphere']=="Northern Hemisphere"]
south_df = city_weather_df[city_weather_df['Hemisphere']=="Southern Hemisphere"]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = north_df['Lat']
y_values = north_df['Max Temp']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(10,40),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Max Temp (F)')
print(f"The r-value is: {rvalue}")
plt.savefig("images/north_lat_maxtemp_regression.png")
plt.show()

This plot shows that temperature and latitude in the northern hemisphere has a strong negative coorelation. The further North you go away from the equater, the lower temperatures drop in the month of September.

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = south_df['Lat']
y_values = south_df['Max Temp']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(-50,80),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Max Temp (F)')
print(f"The r-value is: {rvalue}")
plt.savefig("images/south_lat_maxtemp_regression.png")
plt.show()

This plot shows that temperature and latitude in the southern hemisphere has a strong positive coorelation. The further North you go away from the equater, the lower temperatures drop in the month of September.

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = north_df['Lat']
y_values = north_df['Humidity']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(10,40),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
print(f"The r-value is: {rvalue}")
plt.savefig("images/north_lat_humidity_regression.png")
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = south_df['Lat']
y_values = south_df['Humidity']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(10,40),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
print(f"The r-value is: {rvalue}")
plt.savefig("images/south_lat_humidity_regression.png")
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = north_df['Lat']
y_values = north_df['Cloudiness']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(10,40),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
print(f"The r-value is: {rvalue}")
plt.savefig("images/north_lat_cloudiness_regression.png")
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = south_df['Lat']
y_values = south_df['Cloudiness']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(10,40),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
print(f"The r-value is: {rvalue}")
plt.savefig("images/south_lat_cloudiness_regression.png")
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = north_df['Lat']
y_values = north_df['Wind Speed']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(10,40),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
print(f"The r-value is: {rvalue}")
plt.savefig("images/north_lat_windspeed_regression.png")
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = south_df['Lat']
y_values = south_df['Wind Speed']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(10,40),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
print(f"The r-value is: {rvalue}")
plt.savefig("images/south_lat_windspeed_regression.png")
plt.show()